# 🚀 Building a Startup Advisor System: Step-by-Step Walkthrough


**System Architecture:**
```
Basic Agent → Research Agent → Agent Swarm → Specialized Teams → Complete System
```

Let's get started! 


## 🛠️ Setup and Configuration

First, let's import our libraries and set up the foundation for our agents.


In [8]:
# Core imports for building our agent system
import os
import textwrap
from dotenv import load_dotenv

# Strands framework for AI agents
from strands import Agent
from strands.models import BedrockModel

# Load environment variables (API keys)
load_dotenv()

True

In [12]:
# Configure a Bedrock Model
model = BedrockModel(
    model_id="us.amazon.nova-micro-v1:0",
    region_name="us-west-2"
)

print("LLM: Amazon Nova Micro")
print("Region: us-west-2")


LLM: Amazon Nova Micro
Region: us-west-2


In [13]:
# Utility function for consistent, readable output
DISPLAY_WIDTH = 120

def print_response(text, title=None, width=DISPLAY_WIDTH):
    """Display agent responses in a clean, formatted way"""
    if title:
        print(f"\n🤖 {title}")
        print("─" * width)
    
    # Wrap text to fit our display width
    wrapped = textwrap.fill(str(text), width=width)
    print(wrapped)
    
    if title:
        print("─" * width)

def print_step(step_num, title):
    """Print step headers consistently"""
    print(f"\n{'='*60}")
    print(f"🎯 STEP {step_num}: {title}")
    print(f"{'='*60}")



---

## 🎯 Step 1: Basic Startup Advisor

Let's start with the simplest possible agent - one that can give basic startup advice without any external tools. 

In [14]:
print_step(1, "Basic Startup Advisor")

# Create our first agent with a specific role and personality
basic_advisor = Agent(
    system_prompt="""You are an experienced startup advisor with 10+ years helping entrepreneurs. 
    Provide practical, actionable advice that's encouraging but realistic. 
    Keep responses concise and focus on the most important points.""",
    model=model
)

# Test our basic advisor
startup_idea = "A mobile app that helps people find and share local hiking trails"

print(f"Startup Idea: {startup_idea}")
print("\nAsking for initial advice...")

response = basic_advisor(f"Give me your initial thoughts on this startup idea: {startup_idea}");

print_response(response, "Basic Advisor Response")



🎯 STEP 1: Basic Startup Advisor
Startup Idea: A mobile app that helps people find and share local hiking trails

Asking for initial advice...
Your idea for a mobile app to help people find and share local hiking trails is promising! Here are some key points to consider:

1. **Market Research**: Understand the existing apps in the market. Identify what they do well and where they fall short. This will help you find a unique angle or feature that sets your app apart.

2. **User-Centric Design**: Focus on the user experience. Ensure the app is easy to navigate, with clear information on trails, user reviews, and ratings. Consider features like trail difficulty levels, distance, elevation, and user-generated photos.

3. **Community Engagement**: Build a community around the app. Encourage users to share their experiences, tips, and photos. This can drive organic growth and keep users engaged.

4. **Local Partnerships**: Collaborate with local hiking clubs, parks, and tourism boards. They 

---

## 🎯 Step 2: Research-Powered Agent

Now let's upgrade our agent with web search capabilities using **MCP (Model Context Protocol)**. 

In [21]:
print_step(2, "Research-Powered Agent with Web Search")

# Import MCP libraries for connecting to external services
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient

# Check if we have the required API key
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

if PERPLEXITY_API_KEY:
    # print("✅ Perplexity API key found")
    
    # Create connection to Perplexity search service via Docker
    search_client = MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="docker",
                args=[
                    "run", "-i", "--rm", 
                    "-e", "PERPLEXITY_API_KEY",
                    "mcp/perplexity-ask"
                ],
                env={"PERPLEXITY_API_KEY": PERPLEXITY_API_KEY}
            )
        )
    )
    
else:
    print("No Perplexity API key found")
    search_client = None



🎯 STEP 2: Research-Powered Agent with Web Search


In [18]:
# Function to test our research agent
def test_research_agent():
    """Test the research agent with web search capabilities"""
    
    
    with search_client:
        # Get available search tools
        search_tools = search_client.list_tools_sync()
        
        print(f"🛠️ Available tools: {[tool.tool_name for tool in search_tools]}")
        
        # Create research agent with search capabilities
        research_agent = Agent(
            system_prompt="""You are a market researcher who finds current, factual information.
            Use web search to gather data about markets, competitors, and trends.
            Provide specific data points and cite your sources. Keep responses focused and informative.""",
            model=model,
            tools=search_tools  # This gives the agent search capabilities!
        )
        
        # Test the research capabilities
        query = "hiking app market size and main competitors 2024"
        print(f"Research Query: {query}")
        
        result = research_agent(query);
        
        print_response(result, "Research Agent Results")
        
        return result

# Test our enhanced agent
research_result = test_research_agent()


🛠️ Available tools: ['perplexity_ask', 'perplexity_research', 'perplexity_reason']
Research Query: hiking app market size and main competitors 2024
<thinking> To provide the User with the latest information on the hiking app market size and main competitors for 2024, I will need to use the provided tools to gather current and factual data. The first step will be to use the "perplexity_research" tool to get comprehensive research on the market size and competitors in the hiking app sector for the year 2024.</thinking>


Tool #1: perplexity_research
Here's the comprehensive analysis of the hiking apps market for 2024 based on the latest research:

### Global Hiking Apps Market: Size, Competitors, and Growth Trajectory in 2024

#### Executive Summary
The hiking apps market has seen transformative growth, driven by global participation in outdoor activities, technological advancements, and health consciousness post-pandemic. Market size estimates for 2024 range between $295 million and $1.

---

## 🎯 Step 3: Multi-Agent Collaboration with Swarm

Now we'll introduce **agent swarms** - a powerful way to coordinate multiple AI agents working together. Think of it like having a team of specialists:

- **Lead Agent**: Coordinates the team and delegates tasks
- **Specialist Agents**: Created on-demand for specific research areas


In [19]:
print_step(3, "Multi-Agent Collaboration with Swarm")

# Import the swarm tool for agent coordination
from strands_tools import swarm


def test_swarm_collaboration():
    """Test coordinating multiple agents with swarm"""
    
    # Create coordinator with both search and swarm tools
    with search_client:
        search_tools = search_client.list_tools_sync()
        
        team_coordinator = Agent(
            system_prompt="""You are a Research Team Lead who coordinates market research.
            Use the swarm tool to create specialist agents for different research areas.
            Each specialist agent inherits your search tools and can research their assigned topic.
            Coordinate the team to provide comprehensive market analysis.""",
            model=model,
            tools=search_tools + [swarm]  # Search tools + coordination
        )
        
        tools_available = f"web search + swarm coordination"
    
    print(f"👥 Team Coordinator created with: {tools_available}")
    
    # Give the coordinator a complex task requiring multiple specialists
    complex_task = """
    Research the outdoor recreation app market for a hiking trail app startup.
    
    Create a swarm of 2 specialist agents to research:
    1. Market size, growth trends, and user demographics
    2. Key competitors, their features, and market positioning
    
    Coordinate the research and provide a comprehensive summary.
    """
    
    print("📋 Assigning complex research task to team coordinator...")
    
    team_result = team_coordinator(complex_task);
    
    print_response(team_result, "Team Coordination Results")
    
    return team_result

# Test multi-agent collaboration
swarm_result = test_swarm_collaboration()

print("\n✅ Step 3 Complete: Multiple agents can now work together as a coordinated team!")



🎯 STEP 3: Multi-Agent Collaboration with Swarm
👥 Team Coordinator created with: web search + swarm coordination
📋 Assigning complex research task to team coordinator...
<thinking> To address the User's request, I need to create a swarm of two specialist agents, each focusing on a different aspect of the outdoor recreation app market research. The first agent will research market size, growth trends, and user demographics, while the second will focus on key competitors, their features, and market positioning. After deploying the agents, I will need to coordinate their research efforts and compile a comprehensive summary.</thinking>


Tool #2: swarm

Tool #3: swarm
# Market Research: Outdoor Recreation App Market

As the Research Team Lead, I'll establish a foundation for our hiking trail app market research:

## Market Overview
- The outdoor recreation app market is growing rapidly, with hiking apps seeing increased adoption as outdoor activities surged post-pandemic
- Key players incl

d a proven subscription model ($29.99/year) focusing on community-verified trails
- **Emerging Challengers**: Apps like Cairn and PeakVisor are gaining traction with unique safety features and AR mountain identification
- **Market Gaps**: Limited personalization based on hiker experience level, accessibility needs, and specialized interests (birdwatching, geology, etc.)

## Revenue Model Comparison
- **Freemium dominates**: Most successful apps offer basic functionality free with premium features behind subscription walls
- **Subscription sweet spot**: $3-8/month price point shows optimal conversion rates in outdoor recreation apps
- **Additional revenue streams**: Partnerships with outdoor retailers, local tourism boards, and guided experiences show growing potential

## User Experience Differentiation Opportunities
- **Hyperlocal information**: Trail conditions in real-time remains an underserved need
- **Community verification**: User-generated content with expert verification creat

Based on the comprehensive research conducted by the two specialist agents, here is a consolidated summary of the outdoor recreation app market for a hiking trail app startup:

### Market Overview
- **Market Growth**: The outdoor recreation app market is experiencing significant growth, driven by increased post-pandemic outdoor activity participation. The hiking and trail app segment is projected to reach $4.1 billion by 2027 with an 8.4% CAGR.
- **Key Players**: Major competitors include AllTrails, Gaia GPS, Komoot, and REI's Hiking Project. The market is segmented into free apps with premium features, subscription models, and one-time purchase options.
- **User Demographics**: The primary users are aged 25-45, with secondary growing segments among younger (18-24) and older (45-65) demographics. Cross-platform functionality and integration with wearable technology are essential to meet user expectations.

### Current Trends
- **Social Features**: Integration of social features allowin

---

## 🎯 Step 4: Specialized Agent Roles

Now we'll create **specialized agents** with distinct personalities and expertise:

- **Market Analyst**: Data-focused, finds and analyzes market information
- **Strategic Advisor**: Business-focused, creates actionable strategies

This demonstrates how different system prompts create agents with unique capabilities and perspectives.


In [22]:
print_step(4, "Specialized Agent Roles - Market Analyst & Strategic Advisor")

def create_specialist_team():
    """Create our specialized agent team"""
    
    # Market Analyst - specializes in data gathering and analysis
    market_analyst = Agent(
        system_prompt="""You are a Senior Market Analyst at a top consulting firm.
        Your expertise is gathering market data, analyzing competitors, and identifying trends.
        You provide detailed, data-driven insights with specific numbers and facts.
        Always support your analysis with concrete data points.""",
        model=model,
        tools=([swarm] + (search_client.list_tools_sync() if search_client else []))
    )
    
    # Strategic Advisor - specializes in business strategy and recommendations
    strategic_advisor = Agent(
        system_prompt="""You are a Strategic Business Advisor with 15+ years of experience.
        You excel at translating market research into actionable business strategies.
        Focus on practical recommendations for positioning, pricing, marketing, and growth.
        Provide clear, implementable advice that startups can act on immediately.""",
        model=model
    )
    
    print("👥 Specialist team created:")
    print("   Market Analyst - Data gathering and competitive analysis")
    print("   Strategic Advisor - Business strategy and recommendations")
    
    return market_analyst, strategic_advisor

# Create our specialist team
analyst, strategist = create_specialist_team()



🎯 STEP 4: Specialized Agent Roles - Market Analyst & Strategic Advisor


MCPClientInitializationError: the client session is not running

In [25]:
def create_specialist_team():
    """Create our specialized agent team"""
    
    # Get tools within session if available
    with search_client:  # ✅ Session managed inside function
        search_tools = search_client.list_tools_sync()
        analyst_tools = [swarm] + search_tools
    
    
    # Market Analyst - specializes in data gathering and analysis
    market_analyst = Agent(
        system_prompt="""You are a Senior Market Analyst at a top consulting firm.
        Your expertise is gathering market data, analyzing competitors, and identifying trends.
        You provide detailed, data-driven insights with specific numbers and facts.
        Always support your analysis with concrete data points.""",
        model=model,
        tools=analyst_tools  # ✅ Use tools obtained in session
    )
    
    # Strategic Advisor - specializes in business strategy and recommendations
    strategic_advisor = Agent(
        system_prompt="""You are a Strategic Business Advisor with 15+ years of experience.
        You excel at translating market research into actionable business strategies.
        Focus on practical recommendations for positioning, pricing, marketing, and growth.
        Provide clear, implementable advice that startups can act on immediately.""",
        model=model
    )
    
    print("👥 Specialist team created:")
    print("   Market Analyst - Data gathering and competitive analysis")
    print("   Strategic Advisor - Business strategy and recommendations")
    
    return market_analyst, strategic_advisor

# ✅ Simple call - session handled inside
analyst, strategist = create_specialist_team()

👥 Specialist team created:
   Market Analyst - Data gathering and competitive analysis
   Strategic Advisor - Business strategy and recommendations


In [26]:
def run_specialist_workflow(startup_idea):
    """Run a complete workflow using our specialist agents"""
    
    print(f"🎯 Analyzing startup: {startup_idea}")
    
    # PHASE 1: Market Analysis
    print("\nPHASE 1: Market Analysis")
    print("-" * 50)
    
    research_task = f"""
    Analyze the market for this startup idea: {startup_idea}
    
    Research and provide:
    1. Market size and growth potential
    2. Key competitors and their market share
    3. Target customer segments and their needs
    4. Market trends and opportunities
    
    Use available tools to gather current market data.
    """
    
    market_research = analyst(research_task);
    
    print("✅ Market analysis complete!")
    
    # PHASE 2: Strategy Development
    print("\nPHASE 2: Strategy Development")
    print("-" * 50)
    
    strategy_task = f"""
    Based on this market research, develop a business strategy for: {startup_idea}
    
    Market Analysis:
    {market_research}
    
    Provide strategic recommendations for:
    1. Value proposition and positioning
    2. Target customer segments and personas
    3. Competitive differentiation
    4. Go-to-market strategy
    5. Revenue model and pricing
    6. Key success metrics to track
    """
    
    business_strategy = strategist(strategy_task);
    
    print("✅ Strategy development complete!")
    
    return market_research, business_strategy

# Test our specialist workflow
test_startup = "A mobile app that uses AR to help people learn cooking skills through interactive recipes"

research_results, strategy_results = run_specialist_workflow(test_startup)

# Display results
print_response(str(research_results)[:500] + "...", "Market Research Summary (Truncated)")
print_response(str(strategy_results)[:500] + "...", "Strategic Recommendations Summary (Truncated)")


🎯 Analyzing startup: A mobile app that uses AR to help people learn cooking skills through interactive recipes

PHASE 1: Market Analysis
--------------------------------------------------
<thinking>To analyze the market for the startup idea of a mobile app that uses AR to help people learn cooking skills through interactive recipes, I'll need to conduct comprehensive research using the available tools. Here's my plan:

1. Use the `perplexity_research` tool to gather data on the market size and growth potential of the AR-based cooking education market.
2. Again, use the `perplexity_research` tool to identify key competitors in the AR cooking education market and their respective market shares.
3. Use the `perplexity_research` tool to determine target customer segments for the AR cooking education app and their specific needs.
4. Finally, use the `perplexity_research` tool to identify current market trends and opportunities in the AR cooking education sector.

Once I have gathered all th

tool_name=<perplexity_research> | failed to process tool
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/genai-on-aws/lib/python3.12/site-packages/strands/handlers/tool_handler.py", line 104, in process
    return tool_func.invoke(tool, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/genai-on-aws/lib/python3.12/site-packages/strands/tools/mcp/mcp_agent_tool.py", line 83, in invoke
    return self.mcp_client.call_tool_sync(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/genai-on-aws/lib/python3.12/site-packages/strands/tools/mcp/mcp_client.py", line 183, in call_tool_sync
    raise MCPClientInitializationError("the client session is not running")
strands.types.exceptions.MCPClientInitializationError: the client session is not running
tool_name=<perplexity_research> | failed to process tool
Traceback (most recent call last):
  File "/opt/homebrew/C

 to the user.</thinking>


Tool #4: perplexity_research

Tool #5: perplexity_research

Tool #6: perplexity_research

Tool #7: perplexity_research
<thinking>It appears that there was an error with the `perplexity_research` tool, which prevented me from gathering the necessary market data. Unfortunately, I cannot directly access external databases or APIs to retrieve this information.

However, I can still provide some general insights based on existing knowledge and recent industry reports available up to my last update. If you need the most current and specific data, I recommend consulting market research firms or industry reports directly. Here's what I can share based on the latest available knowledge:</thinking>

1. **Market Size and Growth Potential**:
   - The global augmented reality (AR) market, which includes educational applications like cooking apps, is projected to grow significantly. According to recent reports, the AR market was valued at approximately $39.4 billion in 202

---

## 🎯 Step 5: Content Creation and File Operations

Our final step adds **content creation and file management** capabilities:

- **Content Writer**: Creates marketing materials and campaigns
- **Report Writer**: Generates professional documentation and saves files


In [27]:
print_step(5, "Content Creation and File Operations")

# Import file operation tools
from strands_tools import file_write, editor

def create_content_team():
    """Create agents that can generate content and save files"""
    
    # Content Writer - creates engaging marketing materials
    content_writer = Agent(
        system_prompt="""You are a Creative Marketing Writer specializing in startup marketing.
        You excel at creating compelling copy, campaign ideas, and promotional content.
        Transform business strategies into engaging narratives that resonate with target audiences.
        Focus on benefits, emotional appeal, and clear calls to action.""",
        model=model
    )
    
    # Report Writer - creates professional documentation
    report_writer = Agent(
        system_prompt="""You are a Professional Report Writer who creates executive-level documents.
        Take research and strategy outputs and format them into clean, professional reports.
        Use proper markdown structure, headings, and formatting for easy reading.
        Always save your reports using the file_write toolto save reports. When using file_write, always set force=true to avoid prompts.""",
        model=model,
        tools=[file_write, editor]  # This agent can create and edit files!
    )
    
    print("👥 Content team created:")
    print("   Content Writer - Marketing materials and campaigns")
    print("   Report Writer - Professional documentation")
    
    return content_writer, report_writer

# Create our content team
writer, reporter = create_content_team()



🎯 STEP 5: Content Creation and File Operations
👥 Content team created:
   Content Writer - Marketing materials and campaigns
   Report Writer - Professional documentation


In [28]:
def test_content_creation():
    """Test content creation and file operations"""
    
    # Sample business strategy (normally this comes from previous steps)
    sample_strategy = """
    Business Strategy for AR Cooking Learning App:
    
    • Target Market: Millennials and Gen Z who want to learn cooking (ages 22-40)
    • Value Proposition: "Learn cooking like having a chef in your kitchen"
    • Key Features: AR recipe overlays, real-time guidance, ingredient recognition
    • Revenue Model: Freemium with premium recipes and advanced features
    • Go-to-Market: Social media, cooking influencers, kitchen appliance partnerships
    """
    
    print("Creating marketing content...")
    
    # STEP 1: Content Writer creates marketing materials
    content_task = f"""
    Create compelling marketing content based on this strategy:
    {sample_strategy}
    
    Generate:
    1. A catchy tagline and value proposition
    2. Key marketing messages for social media
    3. Brief description for app store listing
    4. Elevator pitch for investors (30 seconds)
    
    Make it engaging and action-oriented.
    """
    
    marketing_content = writer(content_task);
    
    print("✅ Marketing content created!")
    print_response(str(marketing_content)[:400] + "...", "Marketing Materials Preview")
    
    print("\nCreating professional report...")
    
    # STEP 2: Report Writer formats and saves everything
    report_task = f"""
    Create a comprehensive startup advisory report that includes:
    
    Business Strategy:
    {sample_strategy}
    
    Marketing Materials:
    {marketing_content}
    
    Format this as a professional markdown report with:
    - Executive Summary
    - Business Strategy Overview  
    - Marketing Recommendations
    - Next Steps
    
    Save the report as 'startup_advisory_report.md'
    """
    
    report_result = reporter(report_task);
    
    print("✅ Professional report created and saved!")
    print_response(str(report_result), "Report Generation Result")
    
    return marketing_content, report_result

# Test content creation
marketing_materials, report_status = test_content_creation()



Creating marketing content...
### 1. Tagline and Value Proposition

**Tagline:** "Cook Like a Pro, Anytime, Anywhere!"

**Value Proposition:** "Unlock the secret to mastering cooking with our AR Cooking Learning App. Imagine having a personal chef in your kitchen, guiding you through every step with AR recipe overlays, real-time guidance, and ingredient recognition. Elevate your culinary skills effortlessly and enjoy delicious, home-cooked meals every day!"

### 2. Key Marketing Messages for Social Media

**Instagram:**
- "🌟 Ready to cook like a pro? Our AR Cooking Learning App brings a chef to your kitchen! Try our AR recipe overlays and real-time guidance today! #CookLikeAPro #ARCooking"
- "👩‍🍳👨‍🍳 Struggling with recipes? No more guesswork! Our AR Cooking Learning App recognizes ingredients and guides you step-by-step. Start cooking like a master chef today! #HomeCooked #ARCooking"

**Facebook:**
- "🔍 Ready to transform your kitchen? Our AR Cooking Learning App offers real-time guida

╔══════ File Write Operation ══════╗
║                                  ║
║ Path: startup_advisory_report.md ║
║ Size: 3877 characters            ║
║                                  ║
╚══════════════════════════════════╝

╔═══════════════════════════════════════════════ File Content (md) ═══════════════════════════════════════════════╗
║                                                                                                                 ║
║    1 # Startup Advisory Report for AR Cooking Learning App                                                      ║
║    2                                                                                                            ║
║    3 ## Executive Summary                                                                                       ║
║    4 This report provides a strategic overview and marketing recommendations for the AR Cooking Learning App, t ║
║    5                                                                                                            ║
║    6 ## Business Strategy Overview                                                                              ║
║    7                                                                                                            ║
║    8 ### Business Strategy for AR Cooking Learning App                                                          ║
║    9                                                                                                            ║
║   10 #### Target Market                                                                                         ║
║   11 Millennials and Gen Z who want to learn cooking (ages 22-40)                                               ║
║   12                                                                                                            ║
║   13 #### Value Proposition                                                                                     ║
║   14 "Learn cooking like having a chef in your kitchen"                                                         ║
║   15                                                                                                            ║
║   16 #### Key Features                                                                                          ║
║   17 AR recipe overlays, real-time guidance, ingredient recognition                                             ║
║   18                                                                                                            ║
║   19 #### Revenue Model                                                                                         ║
║   20 Freemium with premium recipes and advanced features                                                        ║
║   21                                                                                                            ║
║   22 #### Go-to-Market                                                                                          ║
║   23 Social media, cooking influencers, kitchen appliance partnerships                                          ║
║   24                                                                                                            ║
║   25 ## Marketing Recommendations                                                                               ║
║   26                                                                                                            ║
║   27 ### 1. Tagline and Value Proposition                                                                       ║
║   28                                                                                                            ║
║   29 **Tagline:** "Cook Like a Pro, Anytime, Anywhere!"                                                         ║
║   30                                                                                                            ║
║   31 **Value Proposition:** "Unlock the secret to mastering cooking with our AR Cooking Learning App. Imagine h ║
║   32                                                                                                            ║
║   33 ### 2. Key Marketing Messages for Social Media   

Do you want to proceed with the file write? [y/*]






Do you want to proceed with the file write? [y/*]






                                                   

KeyboardInterrupt: 

---

## Complete System Demo

Now let's put it all together! This function demonstrates the complete startup advisor system working end-to-end:

1. **Market Research** → Data gathering and analysis
2. **Strategy Development** → Business planning and recommendations  
3. **Content Creation** → Marketing materials and campaigns
4. **Report Generation** → Professional documentation


In [ ]:
def complete_startup_advisory(startup_description):
    """Complete end-to-end startup advisory workflow"""
    
    print("=" * 60)
    print(f"💡 Startup Idea: {startup_description}")
    
    results = {}
    
    # PHASE 1: Market Research  
    print("\nPHASE 1: Market Research & Analysis")
    print("-" * 50)
    
    research_task = f"""
    Conduct comprehensive market research for: {startup_description}
    
    Research:
    1. Market size and growth potential
    2. Key competitors and their positioning
    3. Target customer segments
    4. Market opportunities and challenges
    
    Be specific and data-driven in your analysis.
    """
    
    if search_client:
        with search_client:
            market_research = analyst(research_task);
    else:
        market_research = analyst(research_task);
    
    results['research'] = market_research
    print("✅ Market research completed")
    
    # PHASE 2: Strategy Development
    print("\nPHASE 2: Business Strategy Development")
    print("-" * 50)
    
    strategy_task = f"""
    Develop a comprehensive business strategy based on this research:
    
    Startup: {startup_description}
    Research: {market_research}
    
    Provide detailed recommendations for:
    1. Value proposition and market positioning
    2. Target customer segments and personas
    3. Competitive differentiation strategy
    4. Go-to-market plan and channels
    5. Revenue model and pricing strategy
    6. Key success metrics and milestones
    """
    
    business_strategy = strategist(strategy_task);
    results['strategy'] = business_strategy
    print("✅ Business strategy developed")
    
    # PHASE 3: Marketing Content
    print("\nPHASE 3: Marketing Content Creation")
    print("-" * 50)
    
    content_task = f"""
    Create compelling marketing materials for: {startup_description}
    
    Based on strategy: {business_strategy}
    
    Develop:
    1. Brand positioning and tagline
    2. Key value propositions for different audiences
    3. Social media campaign concepts
    4. Investor pitch summary
    5. App store description
    
    Make it engaging and actionable.
    """
    
    marketing_content = writer(content_task);
    results['marketing'] = marketing_content
    print("✅ Marketing content created")
    
    # PHASE 4: Professional Report
    print("\nPHASE 4: Professional Report Generation")
    print("-" * 50)
    
    report_task = f"""
    Create a comprehensive startup advisory report including all our analysis:
    
    Startup: {startup_description}
    Market Research: {market_research}
    Business Strategy: {business_strategy}
    Marketing Content: {marketing_content}
    
    Format as a professional executive report with:
    - Executive Summary
    - Market Analysis
    - Strategic Recommendations
    - Marketing Plan
    - Implementation Roadmap
    - Appendices
    
    Save as 'complete_startup_advisory_report.md'
    """
    
    final_report = reporter(report_task);
    results['report'] = final_report
    print("✅ Professional report generated and saved")
    
    # Summary
    print("\nSTARTUP ADVISORY COMPLETE!")
    print("=" * 60)
    print("📋 Deliverables generated:")
    print("   Market research and competitive analysis")
    print("   Business strategy and recommendations")
    print("   Marketing content and campaigns")
    print("   Professional advisory report (saved to file)")
    
    return results


##  Demo

Now you can test the complete system with your own startup ideas! Try different types of businesses to see how the agents adapt their analysis and recommendations.


In [ ]:
# Test the complete system with a sample startup
sample_startup = "A subscription service that delivers personalized vitamin packages based on DNA analysis and lifestyle data"

print(f"Testing complete system with: {sample_startup}")
print("\nThis will take a few moments as each agent does their work...")

# Run the complete workflow
all_results = complete_startup_advisory(sample_startup)

# Display preview of results
print("\nResults Preview:")
print("=" * 50)

for phase, result in all_results.items():
    preview = str(result)[:200] + "..." if len(str(result)) > 200 else str(result)
    print(f"\n{phase.upper()}: {preview}")



In [ ]:
# Try with your own startup idea!
# Uncomment and modify this section to test your own ideas

# your_startup_idea = "Enter your startup idea here..."
# 
# print(f"🚀 Analyzing your startup: {your_startup_idea}")
# your_results = complete_startup_advisory(your_startup_idea)
# 
# print("\n🎉 Your startup analysis is complete!")
# print("📄 Check the generated report for detailed recommendations.")

print("💡 Ready to analyze your startup idea!")
print("✏️ Uncomment the code above and add your startup description to get started.")
